In [29]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [30]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint

# MongoBD

In [31]:
client = MongoClient('localhost',27017)

In [32]:
db = client['Mail']

In [33]:
mail_ru = db.mail

# Selenium

In [34]:
ChromeOption = Options()
ChromeOption.add_argument('start-maximized')

In [35]:
driver = webdriver.Chrome('./chromedriver.exe',options = ChromeOption)

In [36]:
driver.get('https://mail.ru/')

In [37]:
user_name = driver.find_element_by_id('mailbox:login').send_keys('study.ai_172')
user_click = driver.find_element_by_id('mailbox:submit').click()
sleep(2)

In [38]:
user_password = driver.find_element_by_id('mailbox:password').send_keys('NextPassword172')
user_click = driver.find_element_by_id('mailbox:submit').click()
sleep(2)

In [39]:
all_mails = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH,'//div[@class = "nav-folders"]//a[@class = "nav__item js-shortcut nav__item_active nav__item_shortcut nav__item_expanded_true nav__item_child-level_0"]'))
    )
all_mails = driver.find_element_by_xpath('//div[@class = "nav-folders"]//a[@class = "nav__item js-shortcut nav__item_active nav__item_shortcut nav__item_expanded_true nav__item_child-level_0"]')
text = all_mails.get_attribute('title')

In [40]:
num = ''
for i in text:
    if i.isdigit():
        num+=i

In [41]:
num = int(num) 

In [42]:
print(f'Всего писем : {num}')

Всего писем : 124


In [43]:
all_links = set()

In [47]:

while True:
    sleep(1)
    mail_scroll = driver.find_elements_by_xpath('//div[@class = "scrollable__container"]//a[@data-id]')
    
    #Собираем все переходы по письмам. Ставим условие на уникальность. 
    for link in mail_scroll:
        all_links.add(link.get_attribute('href'))
    
    action = ActionChains(driver)
    action.move_to_element(mail_scroll[-1])
    action.perform()

    if len(all_links)>=num:
        break

In [48]:
print(f'Собрано писем {len(all_links)}')

Собрано писем 124


In [49]:
today = datetime.today().strftime('%d/%m/%Y').split('/')
today

['12', '07', '2020']

In [50]:
month = {
        1:'января',
        2:'февраля',
         3: 'марта',
         4:'апреля',
         5:'мая',
         6:'июня',
         7:'июля',
         8:'августа',
         9:'сентября',
         10:'октября',
         11:'ноября',
         12:'декабря'
        }

In [51]:
count = 0
for link in all_links:
    driver.get(link)

    
    name_author = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH,'//div[@class ="letter__body"]'))
    )
    
    name_author = driver.find_element_by_class_name("letter-contact").text
    topic = driver.find_element_by_xpath('//div[@class ="thread__header"]//h2').text
    date = driver.find_element_by_class_name('letter__date').text.split(', ')[0]
    time  = driver.find_element_by_class_name('letter__date').text.split(', ')[1]
    all_text = driver.find_element_by_class_name('letter__body')
    
    mail_text = all_text.text
    
    if 'Вчера' in date:
        date = str(int(today[0])-1) +' ' + month[int(today[1])]
    elif 'Сегодня' in date:
        date = str(int(today[0])) +' ' + month[int(today[1])]
    
    
    print(name_author)
    print(topic)
    print(date)
    print(time)
    print(mail_text)
    print(link)
    print('\n' + '*' * 100 + '\n')
    sleep(1)
    try:
        mail_ru.insert_one({
                            '_id':str(link),
                            'name':name_author,
                            'time':time,
                            'date':date,
                            'topic':topic,
                            'text':mail_text
                            
                                   })
    except:
        continue

    

GeekBrains
Подарок на наш День рождения
14 января
15:33
    Привет!
  Сегодня большой праздник.

10 лет назад появился проект «Школа Программирования», который позже превратился в GeekBrains и вошел в Mail.ru Group. Юбилей!

История GeekBrains — это история наших студентов. Каждый студент — наша гордость и вклад в общее будущее.

Поэтому сегодня мы вновь расскажем о студентах. За эти 10 лет кто-то создал игру на 130 000 скачиваний, кто-то ушел в айти в 52 года, кто-то нашел свое призвание и теперь кайфует от работы.

Почитайте праздничную статью и порадуйтесь с нами. А если захотите нас поздравить, пишите в комменты :)
  УРА! Читаю!
          Праздник без подарков — такой себе праздник. Пока коммерческий директор отвлекся и открывает шампанское, мы дарим вам скидку до 70 000 рублей на обучение.
  Выбрать обучение
  Рекомендуем лично вам
    Узнать больше
    Узнать больше
    Узнать больше
  P. S. К 20 января все придут в себя, и скидка закончится.
      Мы в социальных сетях
  Пишите-

GeekBrains
⚡⚡⚡ Новый факультет!
1 апреля
16:00
      Привет!
  Нам часто пишут, что мы учим только современным профессиям и совсем забыли о традициях. А как же те, кто хочет зарабатывать своими руками?

Мы переосмыслили классический подход к рукоделию и сегодня открываем факультет визуализации и крафт-разработки!
  Ни слова больше!
 
Чему вы научитесь?
  Генерировать идеи и креативить на каком-то космическом уровне.
  Выбирать правильные материалы: шерсть, бисер, дерево, фанеру, оргалит и так далее.
  Создавать волшебные вещи и блистать на ярмарках (включая муниципальные).
 
Кому стоит обучаться здесь?
  Маркетологам — мастерить «живой» контент для продвижения брендов.
  Управленцам — делать подарки коллегам и поддерживать дружелюбную атмосферу в коллективе.
  Программистам — развить дополнительные зоны мозга и учиться творчески подходить к задачам.
  Дизайнерам — получить дополнительные знания по композиции.
  Предпринимателям — придумать новый бизнес с миллиардными оборотами.
    Пре

Tproger Live
Вечерний обзор IT-новостей 9 декабря
9 декабря 2019
19:14
Китай наносит ответный удар
Пока администрация Трампа выдавала американским компаниям лицензии для работы с Huawei, китайское правительство решило, что китайские техногиганты и сами с усами. The Financial Times сообщило, что власти Китая планируют за три года отказаться от использования в госучреждениях зарубежных устройств и софта. Всё должно начаться в 2020 году.
Судя по пересказу The Guardian, издание опирается на информацию от инсайдеров из неких компаний по IT-безопасности. Тем о новой импортозаместительной политике сказали некие клиенты из госорганов. Новость неподтверждённая, но выглядит вполне закономерной на фоне американо-китайских отношений в целом и конфликта США и Huawei в частности. 31 мая китайские власти вообще грозились составить зеркальный список неблагонадёжных американских компаний. Тогда конфликт с Huawei только-только вошёл в активную фазу (почитать об этом можно в обзоре за 31 мая).
Суперприло

Mail.ru
Вход с нового устройства в аккаунт
8 июля
19:13
        Вход с нового устройства в аккаунт
 study.ai_172@mail.ru 
      В аккаунт study.ai_172@mail.ru вошли с нового устройства.
  Если вы этого не делали, измените пароль, чтобы обезопасить аккаунт.
  Время 8 июля, 19:13
  Устройство Роботы, Роботы, Москва, Россия
  Посмотреть список устройств
  Узнайте, как защитить аккаунт, на нашем сайте по безопасности.
    Вы получили это письмо, потому что являетесь пользователем Mail‌.ru на основании Пользовательского соглашения.
  Copyright 2020 Mail‌.ru Group, Москва — Все права защищены.
  Служба поддержки      Mail ID
   
https://e.mail.ru/inbox/0:15942247891518516522:0/?back=1&afterReload=1

****************************************************************************************************

GeekBrains
Все об удаленке
7 апреля
11:53
      Привет!
  У нас тут хорошие новости. 14-16 апреля мы проведем для вас серию бесплатных вебинаров об удаленке.

Приходите в онлайн — посидим по дома

Tproger Live
Обзор IT-новостей 23 декабря
23 декабря 2019
10:15
Технологии в законе
«Ведомости» узнали, что представители стран ЕАЭС — Белоруссии, Казахстана и Киргизии — выступили с критикой закона о предустановке российского софта на продаваемые на российском рынке гаджеты. По их словам, этот закон подрывает саму идею о едином с Россией торговом пространстве. Он создаёт препятствия для продажи на территории России техники из стран-участниц ЕАЭС.
Может, так, а может, и нет. Российские чиновники сказали, что никаких препятствий закон не создаёт, это закрепят подзаконные акты — те самые, которые будут указывать, что устанавливать и на какие устройства. Минкомсвязи и Роскомнадзор позаботятся о том, чтобы интересы стран ЕАЭС не были ущемлены.
Окончательное решение о том, есть препятствия или нет, будет принимать позже Евразийская экономическая комиссия.
Если помните, закон о предустановке российского софта уже подписали и приняли, он вступит в силу 1 июля 2020 года. Сейчас никто не знает,

...

GeekBrains
Книги по машинному обучению для новичков
3 апреля
12:30
      Привет!
  Сегодня у нас просто полезная подборка.

Вот список из 12 лучших книг по машинному обучению для новичков. 7 на русском + 5 на английском. Большая часть совсем с нуля, но некоторые книги требуют базовых знаний Python.

Кажется, это идеальный карантин: вы, Шалев-Шварц Шай и наивный байесовский классификатор. Просто... кайф...
  Смотреть список
  P. S. Машинному обучению и Data Science мы обучаем на факультете искусственного интеллекта. Сейчас, кстати, со скидкой 108 000 рублей.
      Скоро вебинары
    Как учиться на удаленке
  3 АПРЕЛЯ, ПТ
19:00 МСК
  Записаться
      Свой веб-сервер на Java за час
  3 АПРЕЛЯ, ПТ
20:00 МСК
  Записаться
      Адаптивная галерея Grid
  4 АПРЕЛЯ, СБ
13:00 МСК
  Записаться
      Как организовать работу команды на удаленке
  4 АПРЕЛЯ, СБ
14:00 МСК
  Записаться
      Мне нужно больше вебинаров!
        Скидки до 40%
  На все факультеты до 13 марта.
    Все факультеты со скидкам

Mail.ru
Вход с нового устройства в аккаунт
8 июля
0:56
        Вход с нового устройства в аккаунт
 study.ai_172@mail.ru 
      В аккаунт study.ai_172@mail.ru вошли с нового устройства.
  Если вы этого не делали, измените пароль, чтобы обезопасить аккаунт.
  Время 8 июля, 00:56
  Устройство Роботы, Роботы, Ростов-на-Дону, Россия
  Посмотреть список устройств
  Узнайте, как защитить аккаунт, на нашем сайте по безопасности.
    Вы получили это письмо, потому что являетесь пользователем Mail‌.ru на основании Пользовательского соглашения.
  Copyright 2020 Mail‌.ru Group, Москва — Все права защищены.
  Служба поддержки      Mail ID
   
https://e.mail.ru/inbox/0:15941589901660897278:0/?back=1&afterReload=1

****************************************************************************************************

GeekBrains
Интересная статистика по зарплатам
24 декабря 2019
11:45
    Привет!
  Наших итогов года в этом декабре не будет (вы и так все знаете), зато будут приятные итоги из мира ваканси

Tproger Live
Обзор IT-новостей 25 декабря
25 декабря 2019
10:40
Apple нас не бросит
Руководитель комитета Госдумы по информационной политике Леонид Левин заявил, что Apple не уйдёт с российского рынка из-за закона о предустановке отечественного ПО. Этот закон СМИ кое-где называют законом против Apple, поскольку корпорация принципиально не предустанавливает на свои гаджеты сторонние приложения. Как докладывали летом «Ведомости», Apple на одном из совещаний уже предупредила чиновников, что такие нормы «вынудят компанию пересмотреть свою бизнес-модель в России».
Причин не беспокоиться насчёт ухода Apple Левин назвал две. Во-первых, у корпорации в России «огромный, миллиардный бизнес», она вряд ли этот бизнес просто так оставит. Во-вторых, по его словам, больше 80 % россиян пользуются Android. По-видимому, это значит, что большинство населения найдёт способ жить без Apple. Несмотря на то, что эти оставшиеся 20 % представляют собой «огромный, миллиардный бизнес».
МТС в деле
МТС сообщила сот

Почта Mail.ru
Новая папка «Письма себе»
18 июня
13:22
  Здравствуйте!
      Встречайте,
«Письма себе»
  Отправляйте важное себе в почту — так оно всегда будет в быстром доступе.
  Включить группировку
      Почему это удобно
  Папка «Письма себе» — аналог заметкам. Отправляйте себе все, что может пригодиться в любой момент:
      Cохраняйте рабочие документы, чтобы иметь к ним быстрый доступ с любого устройства в любое время.
    Составьте список продуктов, которые нужно купить домой
    Cохраняйте важные ссылки, чтобы они не потерялись
  Собрать в папку
    Быстрый поиск
  Верный способ не потерять важную информацию — отправить ее самому себе в почту. Письма всегда под рукой во входящих и в списке папок.
    Как включить группировку писем себе
  1.
Нажмите значок
под списком папок.
  2. Установите галочку «Группировать письма себе».
  Во «Входящих» появится подпапка «Письма себе». Все письма будут попадать туда автоматически.
  Включить группировку
    С наилучшими пожеланиями, команд

Mail.ru
Вход с нового устройства в аккаунт
9 июля
12:17
        Вход с нового устройства в аккаунт
 study.ai_172@mail.ru 
      В аккаунт study.ai_172@mail.ru вошли с нового устройства.
  Если вы этого не делали, измените пароль, чтобы обезопасить аккаунт.
  Время 9 июля, 12:17
  Устройство Microsoft Windows 8, Google Chrome 83, Казань, Россия
  Посмотреть список устройств
  Узнайте, как защитить аккаунт, на нашем сайте по безопасности.
    Вы получили это письмо, потому что являетесь пользователем Mail‌.ru на основании Пользовательского соглашения.
  Copyright 2020 Mail‌.ru Group, Москва — Все права защищены.
  Служба поддержки      Mail ID
   
https://e.mail.ru/inbox/0:15942862550199643571:0/?back=1&afterReload=1

****************************************************************************************************



In [52]:
for i in mail_ru.find({}):
    pprint(i)

{'_id': 'https://e.mail.ru/inbox/0:15790052190194641088:0/?back=1&afterReload=1',
 'date': '14 января',
 'name': 'GeekBrains',
 'text': '    Привет!\n'
         '  Сегодня большой праздник.\n'
         '\n'
         '10 лет назад появился проект «Школа Программирования», который позже '
         'превратился в GeekBrains и вошел в Mail.ru Group. Юбилей!\n'
         '\n'
         'История GeekBrains — это история наших студентов. Каждый студент — '
         'наша гордость и вклад в общее будущее.\n'
         '\n'
         'Поэтому сегодня мы вновь расскажем о студентах. За эти 10 лет кто-то '
         'создал игру на 130 000 скачиваний, кто-то ушел в айти в 52 года, '
         'кто-то нашел свое призвание и теперь кайфует от работы.\n'
         '\n'
         'Почитайте праздничную статью и порадуйтесь с нами. А если захотите '
         'нас поздравить, пишите в комменты :)\n'
         '  УРА! Читаю!\n'
         '          Праздник без подарков — такой себе праздник. Пока '
         'ком

         'пользовательским соглашением. Если вы не хотите получать рассылку, '
         'вы можете отписаться или настроить параметры.\n'
         '  Если письмо отображается некорректно или вы хотите им поделиться, '
         'жмите сюда.\n'
         '  Общество с ограниченной ответственностью «ГикБреинс». ОГРН: '
         '1167746654569. Юридический адрес: 121205, г. Москва, территория '
         'инновационного центра «Сколково», Большой бульвар, д. 42, стр. 1, '
         'этаж 3, пом. 1199, место 5. Телефон 8-800-700-68-41.\n'
         '   ',
 'time': '15:21',
 'topic': 'Письмо от гендира GeekBrains'}
{'_id': 'https://e.mail.ru/inbox/0:15906549170819032600:0/?back=1&afterReload=1',
 'date': '28 мая',
 'name': 'GeekBrains',
 'text': '      Привет!\n'
         '  Если изоляция уже бесит, предлагаем поохотиться на уток. Онлайн, '
         'без жертв и с пользой. На вебинаре «Охота на уток на чистом HTML / '
         'CSS». Бесплатно и легально :)\n'
         '\n'
         'За 1-2 часа

         '  Записаться\n'
         '          Самый последний шанс\n'
         '  Сегодня в 23:59 (МСК) заканчивается наша весенняя акция: скидки до '
         '50 000 рублей на все + шанс выиграть электросамокат + шанс '
         'дополнительно выиграть новенький MacBook Pro.\n'
         '  Я УСПЕЮ\n'
         '    30 марта начинается интенсивный онлайн-курс «MS Project в '
         'управлении».\n'
         '  За 4 урока вы научитесь правильно планировать проекты, '
         'распределять ресурсы, мониторить риски и грамотно оценивать объёмы '
         'работ без долгих вычислений.\n'
         '\n'
         'Записывайтесь сейчас, количество мест ограничено.\n'
         '    Записаться\n'
         '      P.S. Как вам письмо?\n'
         '    Огонь!\n'
         '  Не огонь...\n'
         '      Мы в социальных сетях\n'
         '    Свяжитесь с нами\n'
         '  sales@geekbrains.ru\n'
         '8-800-700-68-41\n'
         'Полезные ссылки\n'
         '  Бесплатные вебинары\n'
       

         '   \n'
         '   \n'
         '  С наилучшими пожеланиями, команда Почты Mail.ru\n'
         ' \n'
         '  Отписаться от рассылки  •  Служба поддержки\n'
         'Mail.ru Group, Москва — Все права защищены.\n'
         ' \n'
         '   \n'
         ' \n'
         ' \n'
         ' ',
 'time': '21:31',
 'topic': 'Ваш ящик разблокирован'}
{'_id': 'https://e.mail.ru/inbox/0:15864213830101465935:0/?back=1&afterReload=1',
 'date': '9 апреля',
 'name': 'GeekBrains',
 'text': '      Привет!\n'
         '\n'
         'Сегодня у нас сразу три статьи: забавная, серьезная и обзор '
         'практических инструментов.\n'
         '\n'
         '\n'
         '😎 Как НЕ устроиться на работу\n'
         '\n'
         'Кажется, мы стали забывать про рубрику «вредные советы». Держите 7 '
         'советов, как гарантированно завалить даже очень удачное '
         'собеседование. Если кратко: лгите, опаздывайте, ругайте бывших '
         'боссов и шутите как ранний Хованский.\n'
     

         '👾 Игры, которые помогут в изучении HTML и CSS\n'
         '\n'
         '🎮 Как стать разработчиком игр\n'
         '  Больше видео тут\n'
         '      🔥 Скидка 40%\n'
         '  На эти и другие факультеты до 29 мая.\n'
         '    Все факультеты со скидками\n'
         '      Ближайшие вебинары\n'
         '    Об информационной безопасности простыми словами\n'
         '  21 МАЯ, ЧТ\n'
         '20:00 МСК\n'
         '  Записаться\n'
         '      Как стать фрилансером?\n'
         '  21 МАЯ, ЧТ\n'
         '20:00 МСК\n'
         '  Записаться\n'
         '      Пишем приложение «Список покупок» на Spring boot\n'
         '  22 МАЯ, ПТ\n'
         '19:00 МСК\n'
         '  Записаться\n'
         '      Как стать разработчиком игр?\n'
         '  23 МАЯ, СБ\n'
         '13:00 МСК\n'
         '  Записаться\n'
         '      Посмотреть все вебинары...\n'
         '  Сейчас!\n'
         '  Все вебинары\n'
         '            Бизнесмены тут?\n'
         '  Если у вас б

         '\n'
         'И как так вышло?\n'
         '  А вот так\n'
         '  Ну и по традиции: с Сергеем можно пообщаться в комментариях, а '
         'выучить Java-разработку — здесь.\n'
         '        Новое видео у нас на YouTube\n'
         '\n'
         'DevOps-инженер — кто это и как им стать?\n'
         '\n'
         'Смотреть сейчас\n'
         '        Вебинары на этой неделе\n'
         '    Метрики тестирования\n'
         '  09 ИЮНЯ, ВТ\n'
         '20:00 МСК\n'
         '  Записаться\n'
         '      Искусственный интеллект vs Программисты\n'
         '  10 ИЮНЯ, СР\n'
         '20:00 МСК\n'
         '  Записаться\n'
         '      Введение в моделирование данных\n'
         '  10 ИЮНЯ, СР\n'
         '20:00 МСК\n'
         '  Записаться\n'
         '      Технология ApplePay\n'
         '  11 ИЮНЯ, ЧТ\n'
         '19:00 МСК\n'
         '  Записаться\n'
         '      Что такое Git и как его использовать?\n'
         '  12 ИЮНЯ, ПТ\n'
         '20:00 МСК\n'
    